In [1]:
import os
import math
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np

from sklearn.model_selection import train_test_split
from keras.models import Sequential, model_from_json
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, ZeroPadding2D, Convolution2D, Dropout
from keras.preprocessing.image import ImageDataGenerator, img_to_array
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard
from sklearn.metrics import f1_score
from PIL import Image

from constant_values import *
from img_preprocessing import *
from predictions import *
from Model_LIDAR import *
from Model_VGG import *
from Model_FCN import *
from Model_UNET import *

%load_ext autoreload
%autoreload 2

Using TensorFlow backend.


Instructions for updating:
non-resource variables are not supported in the long term


### Load full image dataset and augmentation done

In [2]:
TRAINING_SIZE = 1000
seed = np.random.randint(0, 10000)

In [ ]:
tr_imgs = data_augmentation("../Data/training/images/", seed)      #100 images --> 2400 images
gt_imgs = data_augmentation("../Data/training/groundtruth/", seed) #100 images --> 2400 images

In [ ]:
tr_imgs2 = tr_imgs[:TRAINING_SIZE, :, :, :]
gt_imgs2 = gt_imgs[:TRAINING_SIZE, :, :, :]

In [ ]:
# Extract each images into patches (numpy arrays).
img = extract_data(tr_imgs2)  #shape: ((400/16)^2 * TRAINING_SIZE, 16, 16, 3)
gt = extract_labels(gt_imgs2) #shape: ((400/16)^2 * TRAINING_SIZE, 2)

In [ ]:
TRAINING_SIZE

In [ ]:
#model = model_LIDAR((16, 16, 3))
#prediction_training_dir = "MODEL_LIDAR_predictions_training/"
#OUTPUT_FILENAME = "Model_LIDAR"

#model = model_VGG((16, 16, 3))
#prediction_training_dir = "MODEL_VGG_predictions_training/"
#OUTPUT_FILENAME = "Model_VGG"

#model = model_FCN((16, 16, 3))
#prediction_training_dir = "MODEL_FCN_predictions_training/"
#OUTPUT_FILENAME = "Model_FCN"

model = model_UNET((400, 400, 3))
prediction_training_dir = "MODEL_UNET_predictions_training/"
OUTPUT_FILENAME = "Model_UNET"


model.compile(optimizer='adam',loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Step 1: Split into validation and training set     
TEST_SIZE = 0.2 # To be studied
Xtrain, Xtest, gt_train, gt_test = train_test_split(tr_imgs, gt_imgs, test_size=TEST_SIZE, random_state=1)

In [ ]:
Xtrain.shape

In [ ]:
# Step 2: Give weights to classes ?
c_weight = {1: 2., 0: 1.}

In [ ]:
# Step 3: Generate Generators
train_datagenerator = ImageDataGenerator(horizontal_flip=True, vertical_flip=True)
test_datagenerator  = ImageDataGenerator()

train_generator = train_datagenerator.flow(Xtrain, gt_train, batch_size=BATCH_SIZE)
test_generator  = test_datagenerator.flow(Xtest, gt_test, batch_size=BATCH_SIZE)


In [ ]:
# Step 4: Early stop
early_stop_callback = EarlyStopping(monitor='val_acc', min_delta=0, patience=50, verbose=0, mode='max', restore_best_weights=True)

In [ ]:
# Step 5: Training
model.fit_generator(train_generator,
            validation_data=test_generator,
            steps_per_epoch=1000,#math.ceil(TRAINING_SIZE / BATCH_SIZE), #len(train_generator)/16,
            epochs=5,
            callbacks = [early_stop_callback],
            #class_weight=c_weight,
            validation_steps=math.ceil(len(test_generator)/BATCH_SIZE))

### Load model from Json 

In [ ]:
from keras.models import model_from_json
json_file = open('simple_model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)

In [ ]:
train_data_filename = "../Data/training/images/"

if not os.path.isdir(prediction_training_dir):
    os.mkdir(prediction_training_dir)

for i in range(1, TRAINING_SIZE+1):
    #pimg = get_prediction_with_groundtruth(tr_imgs2[i], model)
    #Image.fromarray(pimg).save(prediction_training_dir + "prediction_" + str(i) + ".png")
    oimg = get_prediction_with_overlay(tr_imgs2[i], model)
    oimg.save(prediction_training_dir + "overlay_" + str(i) + ".png")
    sys.stdout.write("\rImage {}/{} is being processed".format(i,TRAINING_SIZE+1))
    sys.stdout.flush()

### Save model to json

In [ ]:
# serialize model to JSON
model_json = model.to_json()
print("1")
with open(OUTPUT_FILENAME + ".json", "w") as json_file:
    json_file.write(model_json)
    
print("2")
# serialize weights to HDF5
model.save_weights(OUTPUT_FILENAME + ".h5")
print("Saved model to disk")

In [ ]:
validation_data_prediction = model.predict_classes(Xtest, verbose = 1)
print("3")
validation_label = []
for e in gt_test:
    print("4")
    if (e[0] == 0):
        validation_label.append(1)
    else:
        validation_label.append(0)
        
validation_label = np.array(validation_label)     
print("F1 score = "+str(f1_score(validation_data_prediction, validation_label, labels=['1'])))